#### 前面两个例子都是分类问题，其目标是预测输入数据点所对应的单一离散的标签。另一种 常见的机器学习问题是回归问题，它预测一个连续值而不是离散的标签

不要将回归问题与 logistic 回归算法混为一谈，logistic 回归不是回归算法， 而是分类算法

### 数据集

本节将要预测20 世纪70 年代中期波士顿郊区房屋价格的中位数，已知当时郊区的一些数 据点，比如犯罪率、当地房产税率等。本节用到的数据集与前面两个例子有一个有趣的区别。 它包含的数据点相对较少，只有506 个，分为404 个训练样本和102 个测试样本。输入数据的 每个特征（比如犯罪率）都有不同的取值范围。例如，有些特性是比例，取值范围为0~1；有 的取值范围为 1~12；还有的取值范围为 0~100

In [17]:
from keras.datasets import boston_housing

(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

###### 有404 个训练样本和 102 个测试样本，每个样本都有13 个数值特征

In [18]:
print(train_data.shape,test_data.shape)

(404, 13) (102, 13)


In [22]:
print(train_data[0])
print(len(train_data))

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
404


In [21]:
print(train_targets[0],len(train_targets))

15.2 404


### 准备数据
- 取值范围差异很大的数据输入到神经网络中，这是有问题的。网络可能会自动适应这种 取值范围不同的数据，但学习肯定变得更加困难
- 对于这种数据，普遍采用的最佳实践是对每 个特征做标准化，即对于输入数据的每个特征（输入数据矩阵中的列），减去特征平均值，再除 以标准差，这样得到的特征平均值为 0，标准差为 1。

In [16]:
import numpy as np

mean = train_data.mean(axis=0)
print(train_data.shape)
print(mean.shape)

train_data -= mean
std = train_data.std(axis=0)
train_data/=std
train_data.shape

(404, 13)
(13,)


(404, 13)

### 构建网络

由于样本数量很少，我们将使用一个非常小的网络，其中包含两个隐藏层，每层有64 个单 元。一般来说，训练数据越少，过拟合会越严重，而较小的网络可以降低过拟合。 

In [24]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsporp',loss='mse',metrics=['mae'])
    return model

网络的最后一层只有一个单元，没有激活，是一个线性层。这是标量回归（标量回归是预 测单一连续值的回归）的典型设置。添加激活函数将会限制输出范围。例如，如果向最后一层 添加 sigmoid 激活函数，网络只能学会预测0~1 范围内的值。这里最后一层是纯线性的，所以 网络可以学会预测任意范围内的值

编译网络用的是 mse 损失函数，即均方误差（MSE，mean squared error），预测值与 目标值之差的平方。这是回归问题常用的损失函数

在训练过程中还监控一个新指标：平均绝对误差（MAE，mean absolute error）。它是预测值 与目标值之差的绝对值。比如，如果这个问题的MAE 等于0.5，就表示你预测的房价与实际价 格平均相差 500 美元

In [25]:
# k折验证
import numpy as np

# 组数
k = 4
# 求出每组的个数
num_val_samples = len(train_data)//k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #',i)
    # 第k个分区的数据
    val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples:(i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
         train_data[(i+1)*num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0) 

model = build_model()
model.fit(partial_train_data,partial_train_targets,
         epochs=num_epochs,batch_size=1,verbose=0)
val_mse,val_mae = model.evaluate(val_data,val_targets,verbose=0)
all_scores.append(val_mae)



processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
Instructions for updating:
Colocations handled automatically by placer.


ValueError: Unknown optimizer: rmsporp